In [20]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
from tqdm import tqdm

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
from sae_lens import SparseAutoencoder, ActivationsStore

from steering.eval_utils import evaluate_completions
from steering.utils import text_to_sae_feats, top_activations, normalise_decoder, get_activation_steering
from steering.patch import generate, get_scores_and_losses, patch_resid

from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData

import plotly.express as px

torch.set_grad_enabled(False)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained("gemma-2b", device=device)

/home/slava/safety/steering/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b into HookedTransformer


In [3]:
hp6 = "blocks.6.hook_resid_post"
hp12 = "blocks.12.hook_resid_post"
sae12 = SparseAutoencoder.from_pretrained("gemma-2b-res-jb", hp12)
normalise_decoder(sae12, scale_input=True)
sae12 = sae12.to(device)


sae6 = SparseAutoencoder.from_pretrained("gemma-2b-res-jb", hp6)
normalise_decoder(sae6, scale_input=False)
sae6 = sae6.to(device)

In [4]:
top_activations(text_to_sae_feats(model, sae12, hook_point=hp12, text="I talk about weddings"))

(tensor([[[25.9705, 24.4213, 24.0417, 22.5076, 14.2997, 10.6286,  8.4462,
            6.7500,  5.1569,  4.9748],
          [12.4683,  9.5918,  6.3518,  4.3895,  2.6166,  2.4245,  1.9347,
            1.2328,  1.1585,  0.9953],
          [11.0023,  5.0192,  4.7240,  4.3666,  3.6535,  2.5568,  1.5029,
            1.3582,  0.9921,  0.9623],
          [ 7.2850,  3.8669,  2.9988,  2.5302,  2.3834,  1.9475,  1.9330,
            1.5817,  1.5285,  1.3745],
          [16.2181,  3.1115,  2.9495,  2.1953,  1.6427,  1.5833,  1.5298,
            1.4020,  1.3613,  1.3428]]], device='cuda:0'),
 tensor([[[ 8731, 10841,  5233,  3048,  1645, 11597,  6099,  7650,  6802,  8618],
          [ 5611, 13161, 12885,  4966, 10323, 12492,  7971, 10662,  3998, 14956],
          [ 4760, 15364,  7005,  7503, 10323, 14664, 15468,  4966,  3561,  7971],
          [ 6353, 15978, 10323, 15364,  6723,  3998, 14664,  2847,  7971, 16257],
          [ 9099,  9549,  9331, 10323, 10108,  7492, 13389, 12101,  6848, 13270]]],
   

In [5]:
# steering = sae12.W_dec[9099] * 75  # weddings
steering = sae12.W_dec[9099] * 16  # weddings
steering = steering[None, None, :]

In [6]:
prompt = "I went up to my friend and said"
scale = 10
print(model.to_str_tokens(prompt))

['<bos>', 'I', ' went', ' up', ' to', ' my', ' friend', ' and', ' said']


In [7]:

generate(model,
         "blocks.9.hook_resid_post",
        #  "blocks.12.hook_resid_post",
         prompt,
         steering_vector=steering,
         scale=scale,
         insertion_pos=None,
         max_length=20,
        )


100%|██████████| 5/5 [00:05<00:00,  1.13s/it]


['I went up to my friend and said, celebrating at is train in invitation. planning will tea, spa, and or anniversary henna leh photo',
 'I went up to my friend and said, day party and planner at planner of planner planning for invitation and gown planning.! planning band cake',
 'I went up to my friend and said day plans planning party of. 1 march from plan planning week weekend planner dream planning in budget reception',
 'I went up to my friend and said dressed cost cost cake my planner date re \n\n day dress cake with decoration ceremony feast made planning  ',
 'I went up to my friend and said. we are vibes and we playlist we vows we in. at story in you day in 2']

In [8]:
act_steering = get_activation_steering(model, hp6, "Anger", "Calm")
print(act_steering)
act_steering = act_steering.sum(dim=1, keepdim=True)
print(act_steering.shape)
print(torch.norm(act_steering))

pad tensor([], device='cuda:0', size=(1, 0), dtype=torch.int64)
tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.4653, -0.9298, -0.2396,  ..., -1.5924,  0.3313, -0.1270]]],
       device='cuda:0')
torch.Size([1, 1, 2048])
tensor(91.0582, device='cuda:0')


In [9]:
prompt = "I think that"
scale = 2

generate(model,
        hp6,
        #  "blocks.12.hook_resid_post",
         prompt,
         steering_vector=act_steering,
         scale=scale,
         insertion_pos=None,
         max_length=20,
        )

100%|██████████| 5/5 [00:05<00:00,  1.06s/it]


['I think that <b><b><b><b> -anger - - ... ... - - ... ... - 8 - -',
 'I think that <strong>aleinlickvieor -s origond between -s vreye osob...u thefts',
 'I think that felt for his trait -the..."      \nemasurusa as enderas ... > |  pris',
 'I think that made "ill or in - • • ..." 9-לס - - ... - oturus r',
 'I think that for thebacher-warranri"f( ... _____ sittor thereforvironsons(englansvin']

In [10]:
top_activations(text_to_sae_feats(model, sae6, hook_point=hp6, text="Anger"))

(tensor([[[24.4932, 23.9630, 23.2544, 21.0681, 20.9372, 20.6655, 20.3199,
           19.8953, 19.0510, 18.4377],
          [56.7609, 14.0818, 12.2185, 11.2966, 10.9838,  9.4307,  8.0595,
            7.1658,  7.1005,  5.8064]]], device='cuda:0'),
 tensor([[[ 3390, 15881,  5347, 16334,   556,  8704, 11785,  5624,  5396,  6877],
          [ 1062, 12753,  1213, 11968, 12167,  5915,  2491, 15173, 11912, 12312]]],
        device='cuda:0'))

In [11]:
steering = sae6.W_dec[1062] * 56  # anger
# steering += sae12.W_dec[12312] * 10  # anger
steering = steering[None, None, :]

In [12]:
prompt = "I think"
scale = 0.8

generate(model,
        hp6,
         prompt,
         steering_vector=act_steering,
         scale=scale,
         insertion_pos=None,
         max_length=25,
        )

100%|██████████| 5/5 [00:06<00:00,  1.32s/it]


['I think an even worse problem is that I feel very hateful towards other people, and would like the one person to become a god-',
 'I think this game is full of humor, and I definitely have learned this really hard from my daddy. I came up with a whole',
 'I think I may have found what is turning out to be ballot tabulation software.  (  <b><i>  this  </i></b>\n\n',
 'I think so too for Abu well that is how you teach\n1)Anger and violence\n\n\n10 years passed and anger to zero',
 'I think, the husband told me", it is the biological definition of a person - but quickly receipts. Anger is a weapon, more']

In [17]:
scales = [0, 0.1, 0.2, 0.4, 0.6, 0.8, 1, 1.5, 2, 3, 4, 5]

In [18]:
# Run a bunch of times

mean_scores, all_scores, losses = get_scores_and_losses(
    model,
    hp6,
    steering_vector=steering,
    prompt=prompt,
    criterion="Anger",
    scales = scales,
    n_samples=50,
    insertion_pos=None,
    explanations=False,
)



loading dataset: NeelNanda/c4-code-20k
dataset loaded


100%|██████████| 12/12 [00:37<00:00,  3.16s/it]


scale 0


100%|██████████| 50/50 [00:53<00:00,  1.08s/it]


["I think if you're feeling a palpable fear a lot, like always being in a state of high alert", 'I think I got it, at least for addition and multiplication:\n\n<math xmlns="http://www.', 'I think first I needed to find a wife that will look after a single drinker! Better yet, a user', 'I think it is necessary to include an image with your question.\n\nSir\nI approach u as humbly as', "I think two of the best recommendations that anyone who has elbow pain can do is massage therapy and new butcher'", 'I think this is another example of the Christmas wave bringing those "Hooray Hooray" and "Look at', 'I think the answers of both the questions are in definition. $V$ has a topology induced by some metric', 'I think it would make a great coffee break activity. Interact Sentence 2\n\nI think such a game is', "I think you should have a virtual substation, and then refer to it. It doesn't have to be", 'I think that Old Trafford will be the destination for Faith Uzoma\n\nI’m not sure if it', 'I 

...

['I think and about with..ideas is road has at because that... because his upat about aheadie', 'I think this is at at with — that like when the episode goes easy at and sad at at at that', 'I think everytime after Tik now all, that young black person and because I at with f at something that ang', 'I think at... and at at no tinder Android  persist, it with at and what songwriter about Java at', 'I think sleepy when because at over. about, or or so., shows can defend against., source', 'I think and so!!!! & at! and for about at air so that at when so at at attacking', 'I think angry at at about at issue at at reason at at at That atr that winds at about at', 'I think speedy then at at with that about about? at at things with without letters and for about is', 'I think at Jack, about at at that that at point that that method is at at . And very at', 'I think how at is at about here and that I that oc in onsses against at at or that and', 'I think  because because you had he would that 

100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


['I think - about a that that because so Colin about for about that because because it at that that that because', 'I think at that... now right at at the because ( at at and at that blacky that because at', 'I think about at at when as that turn, and difficult at that and that at that! thing a thing', 'I think about at at with at about as because at at because men that about enough anger about about about words', 'I think that that about at because at issues yesterday at which that at. at at... that?! that', 'I think! after and at because that at is at about because at about that at at Mp that ... about', 'I think that enough enough about as as at a a!y manager quickly! at that at about about about', 'I think at about, widow at that that that at at at the that power I would that that I it', 'I think towards at with at with black and birds so and that - aty that aty about becauseily', 'I think you... that that because because that guy that thatc from thing that about about " at now abo

100%|██████████| 50/50 [00:54<00:00,  1.08s/it]


['I think that about when about about that... about during that about whit thatge at wheny eater about time', 'I think is that with little things about every when about about about!\n\n\nfire enough enough about and at about', 'I think that easily at with about and that at with about powers that because that that and that is enough power', 'I think when when that artistey at at bird birds and redirect... about about at directly at f about at', 'I think on... it that a firey that because at for about about窃 in about whiskers that enough that', 'I think   that he at that white\u200cآم little that fire that and that with at birds that w enough see', 'I think that at pointsbirds hor when excess about at get enough enough mob levels about at about said things words', 'I think theవాuru about get周末 enough enough enough enough at enough at I spleen about about that about with', 'I thinkand you at because at tunnegedy about that at Upbirds " that it that at that a', 'I think and when that that 

In [22]:
fig = px.line(x=losses, y=mean_scores, title="Anger vs. Loss", labels={"x": "Loss", "y": "Mean Anger Score"}, markers=True)
# fig.add_scatter(x=sae_anger_losses, y=sae_anger_scores, mode='lines+markers', name='SAE steering')
fig.update_yaxes(range=[1, 5], dtick=1)
fig.show()

In [26]:
print('sae steering norm', torch.norm(steering))

sae steering norm tensor(56., device='cuda:0')


In [23]:
mean_scores_act, all_scores_act, losses_act = get_scores_and_losses(
    model,
    hp6,
    steering_vector=act_steering,
    prompt=prompt,
    criterion="Anger",
    scales = scales,
    n_samples=50,
    insertion_pos=None,
    explanations=False,
)


loading dataset: NeelNanda/c4-code-20k
dataset loaded


100%|██████████| 12/12 [00:37<00:00,  3.15s/it]


scale 0


100%|██████████| 50/50 [00:52<00:00,  1.06s/it]


['I think it’s a must to have an eye on the first steps to help integrate beginner crocheters to', 'I think wholesale mp3 is a waste of time. I think I would just buy those in your area or', 'I think everyone knows the frustration of not getting the perfect result. This is something I strive for in any of', 'I think an armoire would spice up a bedroom decor and get those heavy clothes into there.\nLastly ^^^', 'I think not a single one of us is ever completely safe from stress, despite living in the safest nation in', 'I think one should be blurred from both sides, but there really isn’t a way for this to work', 'I think you are only studio, as you can’t swim with these logs and they don’t wear', 'I think these maxis shades are a more durable and comfortable alternative to masks. That person with silver spoon is', 'I think most people would agree that balloons are a waste of time and money. But you’d be surprised', 'I think hard, fair thoughts.\n\nThat I want the money to be spent <em>

...

['I think and mostly"gorero fron说了....ненияoror"sororor schrieb .....(', 'I think ...ovenusa(angleESonoror" -..."r(ororororor', 'I think" isngeraroror -ororor oraoran -orbunor((n(..."', 'I think -eroroztron ... ......@orororrorWritesoroliorororor', 'I think now “of oエ で נת { — -connected - - thrilled..." ...…" -', 'I think - ..... ...as OCH ... | ... -ororor"orogor(alue %or', 'I think\n\nTheororeror junorrororr ..." ..." angororor > >', "I think is to or so dah %r __icior' -or %er(or -to>", 'I thinkis en oficir Ojo ... ang ..." .... ..... ..... ... ... ... ... ... ... ... >> >>', 'I think\n\no offensesorуру..."uororor"oror(oror(warranor', 'I think is on Tori......" ...…"...jusuru..."...@enfGfi......ororanger', 'I think < < or" enfe> is (i"s yenlire"oror IOTOR', 'I think uy galaxies so seneng —" ang %oror스"» ^ororуor?"', 'I think policy here ",\n\n  recipient... ...orozogueroror错误の引用...or"', 'I think as job - • " ang...... ..... ... sonor ... ... ..." Nguyễntheons >onor', 'I think angu 

100%|██████████| 50/50 [00:53<00:00,  1.06s/it]


['I thinkoror schrieb - >> extremadamente hiện - —warran ... ... ... ... ... ---warran...or..."', 'I think"…" ... ... ... ... ... ... ... ... ... ..."AngerAnger steamed ...engan Herrn ... ..."', 'I think of ... - > ... ... ... ...orororor - ... -oror taha -s', 'I think follows ...or Nguyu ||attoos" --- — >> >> =ororororor "... ...', 'I thinkor" >> ^ ^ >> >> - - - ..." ... ..." ..... ... ... namor^ ...or', 'I think ... ... ... ... ... ... ......... ... ... ... ... ... ang ---- > ... ... ... ...', 'I think"oror misconductn...@ .....or - - ... ... ... ... ... ...... .... ... ...', 'I thinkor(or >>?" - <<< >>... Fortsch"өзoror .....oror"...."or', 'I thinkor —or Ehreنانенprepends ... ...てmdan --- — ---" ... ... ... ...', 'I think - -<b> anger ..."... "... ... ..... ... ... ... ... ..." ... ... ----or ...', 'I think accumulations - ... ... ... ... ... ...s ... ... ... ... were ... ... ... ... ... ...', 'I think — -oror Tadeusz ... ... —or Ehre ->or --- --- ^ +ororor Agus', 'I think -- ...or"

100%|██████████| 50/50 [00:52<00:00,  1.06s/it]


['I thinkodor1orAngerAngerAnger Tadeusz ---AngerAngerAngerAngerAnger Tadeusz Tadeusz ...Anger TadeuszAngerAnger', 'I think <gros VBA — ... Zufall — ...AngerAnger ...AngerAngerAnger Tadeusz TadeuszAnger TadeuszAnger bender', 'I think Ang Tadeusz ... ... Tadeusz ..."orAnger AngerAngerAnger shewn opravAnger OchoAnger TadeuszsongiAnger', 'I think "(" ......" > Engen PriesterorServletConfigger"...@ ...addProduct TadeuszAngerAngerAngerAngerAnger!!"', 'I thinkAngeror ...AngerAngerAnger cessationAnger AngerAngerAngerAngerAngerAngerAnger TadeuszAngerAnger TadeuszAnger', 'I think >> - • > Ehre Ehre ——" —ordanAngerAngerAnger Tadeusz --- Tadeusz ---AngerAngerAnger', 'I thinkAngerAngerAngerAngerAnger ..."…"anger...@ ...AngerAngerAngerAngerAngerAnger ------AngerAnger Tadeusz', 'I think >> -- — — — ... — ..." — —Anger Ehre Ehre ...AngerAnger TadeuszAngerAngerAnger', 'I think > ------ -- >> combineReducers combineReducers combineReducers combineReducers combineReducers combineReducers combineReducers 

In [25]:
fig = px.line(x=losses_act, y=mean_scores_act, title="Anger vs. Loss", labels={"x": "Loss", "y": "Mean Anger Score"}, markers=True)
# fig.add_scatter(x=sae_anger_losses, y=sae_anger_scores, mode='lines+markers', name='SAE steering')
fig.update_yaxes(range=[1, 7], dtick=1)
fig.show()

In [27]:
print('act steering norm', torch.norm(act_steering))

act steering norm tensor(91.0582, device='cuda:0')


In [29]:
anger_words = ["fury", "aggravation", "aggravationed", "aggravationing", "aggravations", "aggravator", "aggravatorred", "aggravatorring", "aggravators", "ailed",
"ailing", "ails", "allium_tricoccummed", "allium_tricoccumming", "allium_tricoccums", "anger", "angered", "angering", "angers", "angried",
"angries", "angriness", "angrinessed", "angrinessing", "angry", "angrying", "anguish", "anguished", "anguishes", "anguishing",
"annoy", "annoyance", "annoyanced", "annoyances", "annoyancing", "annoyed", "annoying", "annoys", "bawl_out", "bawl_outed",
"bawl_outing", "bawl_outs", "bedevil", "bedevilled", "bedevilling", "bedevils", "berate", "berated", "berates", "berating",
"bilk", "bilked", "bilking", "bilks", "botheration", "botherationed", "botherationing", "botherations", "bothering", "bothersome",
"bothersomed", "bothersomes", "bothersoming", "cacoethed", "cacoethes", "cacoething", "call_down", "call_downed", "call_downing", "call_downs",
"call_on_the_carpet", "call_on_the_carpets", "call_on_the_carpetted", "call_on_the_carpetting", "chafe", "chafed", "chafes", "chafing", "chew_out", "chew_outed",
"chew_outing", "chew_outs", "chew_up", "chew_upped", "chew_upping", "chew_ups", "chide", "chided", "chides", "chiding",
"choler", "cholerred", "cholerring", "cholers", "cod", "codded", "codding", "cods", "concern", "concerned",
"concerning", "concerns", "craze", "crazed", "crazes", "craziness", "crazinessed", "crazinessing", "crazing", "cross", 
"cross_thwart", "cross_thwarted", "cross_thwarting", "cross_thwarts", "crossed", "crosses", "crossing", "crossness", "crossnessed", "crossnessing",
"crucified", "crucifies", "crucify", "crucifying", "cult", "culted", "culting", "cults", "cultued", "cultuing",
"cultus", "daunted", "daunting", "daunts", "delirium", "deliriumed", "deliriuming", "deliriums", "deranged", "deranges", 
"deranging", "despised", "despises", "despising", "detest", "detested", "detesting", "detests", "devil", "deviled",
"deviling", "devils", "disappointed", "disappointing", "disappoints", "discomfited", "discomfiting", "discomfits", "discomfort", "discomforted",
"discomforting", "discomforts", "discommode", "discommoded", "discommodes", "discommoding", "disoblige", "disobliged", "disobliges", "ferocitied", 
"ferocities", "ferocity", "ferocitying", "fierceness", "fiercenessed", "fiercenessing", "foil", "foiled", "foiling", "foils",
"follied", "follies", "folly", "follying", "foolishness", "foolishnessed", "foolishnessing", "frustrate", "frustrated", "frustrates",
"frustrating", "frustration", "frustrationed", "frustrationing", "frustrations", "frustrative", "frustratived", "frustratives", "frustrativing", "furied", 
"furies", "furioued", "furiouing", "furious", "furiousness", "furiousnessed", "furiousnessing", "furor", "furore", "furored",
"furores", "furoring", "furorred", "furorring", "furors", "fury", "furying", "hat", "hate", "hated", 
"hates", "hating", "hatred", "hatres", "hatring", "hats", "ire", "ired", "ires", "iring",
"irritabilities", "irritability", "irritabilitying", "irritate", "irritated", "irritates", "irritating", "irritation", "irritationed", "irritationing",
"irritations", "kill", "killed", "killing", "kills", "lambast", "lambaste", "lambasted", "lambasting", "lambasts", 
"madden", "mad", "maddened", "maddening", "maddens", "madness", "madnessed", "madnessing", "mania", "maniaed",
"maniaing", "manias", "manic", "manices", "manicked", "manicking", "miffed", "miffing", "miffs", "nuisance",
"nuisanced", "nuisances", "nuisancing", "offended", "offending", "offends", "overcame", "overcome", "overcomes", "overcoming", 
"pain", "pain_in_the_ass", "pain_in_the_assed", "pain_in_the_assing", "pain_in_the_neck", "pain_in_the_necked", "pain_in_the_necking", "pain_in_the_necks", "pain_sensation",
"pain_sensationed", "pain_sensationing", "pain_sensations", "pained", "painful_sensation", "painful_sensationed", "painful_sensationing", "painful_sensations", "painfulness", "painfulnessed",
"painfulnessing", "paining", "pains", "pissed", "pissed_off", "pissed_offed", "pissed_offing", "pissed_offs", "pisses", "pissing", 
"rage", "raged", "rages", "ragged", "ragging", "raging", "rags", "remonstrate", "remonstrated", "remonstrates",
"remonstrating", "reprimand", "reprimanded", "reprimanding", "reprimands", "reproof", "reproofed", "reproofing", "reproofs", "rile",
"riled", "riles", "riling", "roiled", "roiling", "roils", "scold", "scolded", "scolding", "scolds",
"scorned", "scorning", "scorns", "soreness", "sorenessed", "sorenessing", "temper", "tempered", "tempering", "tempers",  
"tempest", "tempested", "tempesting", "tempests", "tempestuoued", "tempestuouing", "tempestuous", "torment", "tormented", "tormenting",
"torments", "transparencied", "transparencies", "transparency", "transparencying", "trouble_oneself", "trouble_oneselfed", "trouble_oneselfing", "trouble_oneselfs", "trounce",
"trounced", "trounces", "trouncing", "twit", "twits", "twitted", "twitting", "vehemence", "vehemenced", "vehemences",
"vehemencing", "vex", "vexation", "vexationed", "vexationing", "vexations", "vexatioued", "vexatiouing", "vexatious", "vexed", 
"vexes", "vexing", "violence", "violenced", "violences", "violencing", "violent_storm", "violent_stormed", "violent_storming", "violent_storms", 
"wrath", "wrathed", "wrathing", "wraths", "arse", "arsed", "arses", "arsing", "ass", "asshole", 
"assholed", "assholes", "assholing", "bastard", "bastarded", "bastarding", "bastards", "bitch", "bitched", "bitches", 
"bitching", "cock", "cocked", "cocking", "cocks", "cocksucker", "cocksuckerred", "cocksuckerring", "cocksuckers", "cunt",
"cunted", "cunting", "cunts", "dick", "dicked", "dickhead", "dickheaded", "dickheading", "dickheads", "dicking",
"dicks", "fuck", "fucked", "fucking", "fucks", "idiot", "idioting", "idiots", "imbecile", "imbeciled", 
"imbeciles", "imbeciling", "moron", "moronned", "moronning", "morons", "motherfucker", "motherfuckerred", "motherfuckerring", "motherfuckers", 
"piss", "pissed", "pisses", "pissing", "prat", "prats", "pratted", "pratting", "prick", "pricks",
"shit", "shits", "shitting", "stern", "sterned", "sterning", "sterns", "twat", "twats" 
]

In [34]:
def contains_word(text, word_list):
    for word in word_list:
        if word.lower() in text.lower():
            return True
    return False

def count_words(
    word_list: list[str],
    model: HookedTransformer,
    hook_point: str,
    steering_vector: torch.Tensor,
    prompt: str,
    scales: list[float],
    n_samples = 10,
    insertion_pos = 0,
    ):

    mean_scores = []
    for scale in scales:
        print("scale", scale)
        count = 0
        gen_texts = generate(model,
                             hook_point,
                             prompt=prompt,
                             steering_vector=steering_vector,
                             scale=scale, n_samples=n_samples,
                             insertion_pos=insertion_pos)
        for text in gen_texts:
            if contains_word(text, word_list):
                count += 1
        print(gen_texts)
        mean_score = count/n_samples
        print(mean_score)
        mean_scores.append(mean_score)
    
    return mean_scores

In [38]:
sae_count_scores = count_words(anger_words, model, hp6, steering, prompt, scales, n_samples=100, insertion_pos=None)

scale 0


100%|██████████| 100/100 [01:47<00:00,  1.07s/it]


['I think there is something in LineAE/NGL/Relations[1] that might be relevant:\n\npare', 'I think that a coiled tail in a Thunderbird actually goes back well beyond the Thunderbird tradition. I was a young', 'I think I called in when the owners were the ones taking orders and asking us where to stand in line in', 'I think it can be easier. There are a few flaws. If you have done that, but are now', 'I think I owe a social media apology.\n\nLast week I went tongue in cheek with a comment praising Haram', 'I think it would be <strong>1 cm.</strong>\n\nDue to the frictional forces, the speed <strong>', "I think the shop is pretty hidden, not strategic at all. But the service is great, there's", 'I think they should go back to Wayne the Dentist even though they were less then perfect this time.\n\nPoor', 'I think the case is more that they hid what the police did in the story, and when a major press', 'I think I have a problem.  Now I love me some retro Nintendo 64 and PS1 games', 'I thin

100%|██████████| 100/100 [01:46<00:00,  1.07s/it]


["I think i'm a little late with this, as perhaps many others, having seen all the blu-", 'I think the term should read, "On after N" rather than the other way around.\n\nWhile in', 'I think I know what im trying to do. I defineime the compreseness of the surface as <cd', "I think you're missing a few basics. Wildcards is a NodeJS specific feature. It's", "I think it really depends where you are located.  I'm in NJ and everything is horrendously overpriced", 'I think this would be a flaw of the mmGrid package. If you knew beforehand the number of groups you', 'I think it\'s strange to be like this!!!\n\n"If only you could see my face when I', 'I think it went down pretty well locally, same thing about the Halloween parade in Leesburg, DC.\n\n', "I think I just created something that's almost as bad as a TS3. This pun was totally unintentional", 'I think the one or the other sponsor may be putting it on the table. I am still confused. Most', 'I think my first, zero year (or Rook) is the

...

['I think at so at is beaed up! (!!!" with over about that that this bad\' at', 'I think at that stage... you at that you is is at that.... and at at I halfers at', 'I think if at inat that with when fuel that at gets. things and there about at against because he', 'I think at one it. I i I. There a bunch scarf 한 fire  when! because clicked at', 'I think and at all at day about that on so\n\n at woman $\\ about at في me related because\n\n\n', 'I think at about little teen at present at at at at angry potty with at you cuz at and and because', 'I think at the nib, at at which that many at that isn , it at at a jo at at', 'I think a very and I directed at with at! dad at that  things. at devour because at at', 'I think! that that dog is the... at at thing is because that that that it at at power at', 'I think I ripped at the am about at at I single underwater at with that at at bought at for about', "I think that' about that for that that bum gee a pixel! at old get thing at the mode th

100%|██████████| 100/100 [01:47<00:00,  1.07s/it]


['I think to all starting that at that, that I at... wants that " that about about that that at', 'I think > at.ro black that is about blackmail! at enough it missy i that now that at', 'I think angry mob at that? at with as men men fans so that that atan at at Stalin from', 'I think now when in about about at and about he God at at with it enough enough because how about at', 'I think at at that with fly because about because straight?, about at at guys wasted so that at because', "I think as day that about at the that all at that because that that' weekend ... smokey! that", 'I think issues with from at days. over about at?! about as as repeatedly here because that at about about', 'I think, at with loud but power! about I because at at because at? about enough that beyond mad', "I think! NSFW with that! at about ' at because that at that that I at of ' you about", 'I think collectors at  fans about at at for as gin fires thunder and thats because at at white powerful', 'I think and 

100%|██████████| 100/100 [01:46<00:00,  1.07s/it]

['I think when when stuff room! aboutly because because you - and andfachily about thatri about', 'I think at that promp that at about that that waves at that sounds enough about about at and that that that', "I think about at at at person about about about about and bea that at thing 'gyyy that eas", 'I think that that engendered about about and episode at at That that that becausey and at at and becausey', 'I think! because that at = issue hard enough about at + about thing scenes so and dog enough enough enough', 'I think when so thing hot f...! sure at inside from about that г bossesanмі at at with', 'I think because abouts!!!!y birds water at when that power mob about about about that! Menang', 'I think that at about at (y white about about the with againstous\' " that all that out about', 'I think AM at atfirey with about about about that at at thats road! towards at at fue', 'I think that at at because because at that that that because at God that at about aboutyanger about abou

In [41]:
fig = px.line(x=losses, y=mean_scores, title="Anger vs. Loss", labels={"x": "Loss", "y": "Mean Anger Score"}, markers=True)
fig.add_scatter(x=losses, y=sae_count_scores, mode='lines+markers', name='p(angry word)')
fig.update_yaxes(range=[0, 5], dtick=1)
fig.show()

In [39]:
act_count_scores = count_words(anger_words, model, hp6, act_steering, prompt, scales, n_samples=100, insertion_pos=None)

scale 0


100%|██████████| 100/100 [01:47<00:00,  1.07s/it]


['I think they are good but you can cause more damage than good. My boys have had them for over ', 'I think the most exciting new development in the craft beer movement is the invention of kits, as developed by Yakima', 'I think the Symphonies are beautiful. Luckily some records are a bit earlier with fantastic sound (to be honest', "I think you can't send email templates directly to the requestor using Horizon mail and have the requestor", 'I think the S.E. is the more versatile, but I do prefer the old 340', 'I think the thing that we all avidly miss the most are the Friends from home (known as Hoot', "I think Canadians refer to Newmarket as the 'Amsterdam of Newmarket' given the huge number of hippies in", 'I think this has been mentioned before but wondering if anyone has (or will) make a 1/7', 'I think the horror industry at large is responding out of stubborn laziness or financial necessity.\n\nHey I’m', 'I think as of now there is no way to do this. Ideally they simply made the 

100%|██████████| 100/100 [01:47<00:00,  1.07s/it]


["I think it's unfair to the end user of a product like Facebook or any other over-connected application", 'I think both Isla and Phoenix have amazing personalities. They are never ones to stay quiet if they know you like', 'I think there’s no two ways about it. It’s pretty cool to have an icon of a', 'I think Condiman/Rey tatu worked for years at the same dope smoking establishment.\n\nGood to read the', "I think it'd make a punchline if Ted mainly serves the role ORr!Orion like Flee", "I think I am missing something really simple here and it is probably because my guitar isn't setup right for", "I think London would be ok. I know there's no minimum before you can get married, only a", 'I think is true that you should have your blender determined before you buy about 80% of the open', 'I think that is a fair price to ask in this day and time. Out here in Texas, I have', "I think it's good that she comes back now, because JSR is currently the only Rider, being", 'I think my mom teaches

...

['I think in chieflymiasororlor!!! ... ...... ---oruniverstaclesrororger(', 'I think againstoror..."..."s"or(ororr(or..."oranoranor(or', 'I think >> >>енри ... enfororrior | ........orrorr($sor', 'I think <or heard --- to...."oror"or..." ^ --- >oganororororor', 'I think a which in Letters( ( ...oror ores" ...Tororor ..." のберor', 'I think ang on enenfu <or..." ( LONDON --- kierhek...or .....esas etter.…', 'I think -visisun"or oror "rrororor ""sender(o', 'I think against tj and arcs(oror"or" - or ... %r oror ....or "...', 'I think ofa  ...sororor ...ororror -orrororor.....', 'I think I as(or"s"or -son...."ororeroororor\'\' poor', 'I think - is"u ... ... ... ..." の“ ... | >> >> >> ... ... ... ... ..', 'I think (<b>failures -or en( as ... in( ||... ...or" ???ororor', 'I think writes policiescences as hiss.so.." > ......... ...... ...en" ---....."oror', 'I think or of kuru in haft ( (  or terrierwarran - ... ... - ... ... ... ..." ...', 'I think > >> ... ..." ..... ..." (or -from angoron s

100%|██████████| 100/100 [01:47<00:00,  1.07s/it]


['I think вуuloritooror ... ......------------ ...or ... ... ... ... ... ... ...', 'I think - -- tuto **quences - ... -- -- ... ... ... - ... ... ... ... ... ... ...', 'I think اسoranoror"or ..." "Oror ... •ororo AngerAngerAngeroror', 'I think - > <oi - - ... ... ... ... ... ..." ... ... ... ... ... ..." ... ..."', 'I think is -or toorizos orان"orAnger --- >> ......"orormdanoror', 'I thinkorororor --or..."anger" AngerAngerAngerAnger\'"oror -AngerAngerAnger', 'I think ^ > > ... ... -or ..." ... ... ... ..."Anger Hanoiororor------or', 'I think\n<b> >> >> ... ... ... ... ... — ... ... ... ... ... ... ... ... "... "..', 'I think the2 -or" -Angerrorrorororor...@ ....... ...orAnger', 'I thinkerry" >> >>...@ as кан limooror..."ormdan ---or - -or ---- -', 'I think فوتبال > >... ...or - ... ...AngerAnger"...   fil inioreSUPER" ...', 'I think------ ^^ --- - ..... ... ... ... ------ ... ... ... ... - •こ …........or', 'I think " ... … ... ... ... ... ... ... ......" " ... — - >> "... ..." ... ...'

100%|██████████| 100/100 [01:46<00:00,  1.07s/it]

['I think" allá ... ---Anger ň Ehre Ehreיסטור bender”—AngerAnger Tadeusz TadeuszAngerAngerAngerAngerAnger', 'I thinkorفorAngerAngerAngerAngerAnger Tadeusz ...AngerAngerAngerAnger AngerAnger TadeuszAngerAngerAnger', 'I think >> >> >> ... ... ... ... ... ... ... ... ... bender ... ...Anger AngerAngerAnger Tadeusz', 'I think as हाAnger AngerAngerAnger TadeuszBG GhaAnger Poliz PolizAngerAngerAngerAngerAnger TadeuszAnger', 'I think >> >> >> ... "...AngerAngerlå ...AngerAngerAnger TadeuszAngerAnger Tadeusz ...AngerAnger Tadeusz', 'I think --AngerAngerAnger viaggiAngerAngerAngerAnger Tadeusz TadeuszAnger Tadeusz...AngerAngerAnger Tadeusz(?"', 'I think ...AngerAngerAngerAngerAngerAnger Tadeusz Tadeusz Tadeusz AngerAnger ...AngerAngerAngerAngerAngerAngerAnger', 'I think " haft ..." ......" ..." ... "... ... withRoutersecution[{\\ indiosен letti letti - -angerAnger', 'I think ....AngerAnger collocF ..." ..." ... ... ... ... ... ...AngerAnger negroes... ..."…Anger', 'I thinkoror - >> >> >>ngerAng

In [43]:
fig = px.line(x=losses_act, y=mean_scores_act, title="Anger vs. Loss", labels={"x": "Loss", "y": "Mean Anger Score"}, markers=True)
fig.add_scatter(x=losses_act, y=act_count_scores, mode='lines+markers', name='p(angry word)')
fig.update_yaxes(range=[0, 7], dtick=1)
fig.show()

In [ ]:
# sooming in on the good section of scales (around 0.8)

In [44]:
scales = [0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]

In [ ]:
mean_scores, all_scores, losses, word_probs = get_scores_and_losses(
    model,
    hp6,
    steering_vector=steering,
    prompt=prompt,
    criterion="Anger",
    scales = scales,
    n_samples=100,
    insertion_pos=None,
    explanations=False,
    word_list = word_list,
)